In [ ]:
import pandas as pd
from tqdm import tqdm

def clean_name(name):
    endIndex = name.find("#")
    if endIndex != -1:
        name = name[0:endIndex]
    name = name.strip()
    
    return name

# Run to combine inspections and violations data

In [ ]:
# Have downloaded files in current directory
inspections = pd.read_csv("Environmental_Health_Restaurant_and_Market_Inspections.csv", encoding = "latin1")
violations = pd.read_csv("Environmental_Health_Restaurant_and_Market_Violations.csv")

In [ ]:
# Combine inspections dataset with violations dataset
def combine_data(inspections, violations):
    finalInspections = inspections.copy()
    # Loop through each inspection
    for i in tqdm(range(len(finalInspections))):
        inspectionIndex = finalInspections.index[i]
        # Find associated violations by serial number
        serialNumber = finalInspections.loc[inspectionIndex, "SERIAL NUMBER"]
        restaurantViolations = violations.loc[violations["SERIAL NUMBER"] == serialNumber]
        # Add violations to inspections data
        for j in range(len(restaurantViolations)):
            index = restaurantViolations.index[j]
            status = restaurantViolations.loc[index, "VIOLATION STATUS"]
            code = restaurantViolations.loc[index, "VIOLATION CODE"]
            description = restaurantViolations.loc[index, "VIOLATION DESCRIPTION"]
            points = restaurantViolations.loc[index, "POINTS"]
            
            finalInspections.loc[inspectionIndex, "VIOLATION STATUS " + str(j)] = status
            finalInspections.loc[inspectionIndex, "VIOLATION CODE " + str(j)] = code
            finalInspections.loc[inspectionIndex, "VIOLATION DESCRIPTION " + str(j)] = description
            finalInspections.loc[inspectionIndex, "POINTS " + str(j)] = points
    
    return finalInspections

In [ ]:
finalData = inspections.copy()    
finalData = finalData.drop(columns = ["OWNER ID", "RECORD ID", "FACILITY ID", "EMPLOYEE ID"])

adjustedFinalData = combine_data(finalData, violations)

In [ ]:
adjustedFinalData.to_csv("health_inspections.csv", index = False)

# Run to generate file to store ratings

In [ ]:
data = pd.read_csv("./health_inspections.csv", low_memory = False)
queries = []
for i in tqdm(range(len(data))):
    inspection = data.loc[i].copy()
    facility = clean_name(inspection["FACILITY NAME"]).strip()
    address = inspection["FACILITY ADDRESS"].strip()
    city = inspection["FACILITY CITY"].strip()
    state = inspection["FACILITY STATE"].strip()
    zipCode = inspection["FACILITY ZIP"].strip()
    
    query = f"{facility} {address}, {city}, {state} {zipCode}, USA"
    
    queries.append(query)

ratings_data = pd.dataFrame({"query": queries, "id": ["Unknown"] * len(queries), "rating": ["Unknown"] * len(queries)})
ratings_data.to_csv("ratings_data.csv", index = False)

# Run to combine inpsections data with collected ratings

In [ ]:
data = pd.read_csv("./health_inspections.csv", low_memory = False)
for i in tqdm(range(len(data))):
    inspection = data.loc[i].copy()
    facility = clean_name(inspection["FACILITY NAME"]).strip()
    address = inspection["FACILITY ADDRESS"].strip()
    city = inspection["FACILITY CITY"].strip()
    state = inspection["FACILITY STATE"].strip()
    zipCode = inspection["FACILITY ZIP"].strip()
    
    query = f"{facility} {address}, {city}, {state} {zipCode}, USA"
    
    data.loc[i, "QUERY"] = query
data.to_csv("health_inspections.csv", index = False)

In [ ]:
data = pd.read_csv("./health_inspections.csv", low_memory = False)
ratings = pd.read_csv("./ratings_data.csv")

In [ ]:
indices = ratings.loc[ratings["rating"] != "Unknown"].index

for i in tqdm(indices):
    query = ratings.loc[i, "query"]
    rating = ratings.loc[i, "rating"]
    dataIndices = data.loc[data["QUERY"] == query].index
    data.loc[dataIndices, "RATING"] = rating

In [ ]:
data.to_csv("health_inspections.csv", index = False)